# Import Appropriate Modules

In [ ]:
import copy
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import sklearn
import random
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import talib as ta
import lightgbm as lgb
from hyperopt import fmin, hp, partial, tpe, Trials

#silence warnings output...
import warnings
warnings.filterwarnings("ignore")

#following for the ML part...setting a seed meant that multiple executions will yield the same result.
random_state = 42
random.seed(random_state)

# Paths and Load CSV (Modify Path Here)

In [ ]:
data_path1 = '''/kaggle/input/hku-qids-2023-quantitative-investment-competition/qids_package/'''
data_path2 = '''/kaggle/input/hku-qids-2023-quantitative-investment-competition/'''
fn_fund = '''first_round_test_fundamental_data.csv'''
fn_market = '''first_round_test_market_data.csv'''
fn_train_fund = '''first_round_train_fundamental_data.csv'''
fn_train_market = '''first_round_train_market_data.csv'''
fn_train_returns = '''first_round_train_return_data.csv'''

df_fund = pd.read_csv(fn_fund)
df_market = pd.read_csv(fn_market)
df_train_fund = pd.read_csv(fn_train_fund)
df_train_market = pd.read_csv(fn_train_market)
df_train_returns = pd.read_csv(fn_train_returns)

df_list = [df_fund, df_market, df_train_fund, df_train_market, df_train_returns]
df_names = ['df_fund', 'df_market', 'df_train_fund', 'df_train_market', 'df_train_returns']
for df, df_name in zip(df_list, df_names):
    print(f"Columns of {df_name} are: {list(df.columns)}")

In [ ]:
#To Create columns containing data column of "date_time"
def interval_split(dt):
    #for date_time column of "sXXdXXpXX"
    f2, f3 = dt.find("d"), dt.find("p")
    return [int(dt[1:f2]), int(dt[f2+1:f3]), int(dt[f3+1:])]
def date_split(dt):
    #for date_time column of "sXXdXX"
    f2 = dt.find("d")
    return [int(dt[1:f2]), int(dt[f2+1:])]
def add_interval(df):
    df_interval_data = np.vstack(df.date_time.apply(lambda x: interval_split(x)))
    df[["asset", "day", "interval"]] = df_interval_data
    return df
def add_date(df):
    df_date_data = np.vstack(df.date_time.apply(lambda x: date_split(x)))
    df[["asset", "day"]] = df_date_data
    return df
print("Ran")

# Features Library

In [ ]:
def add_period(df):
    #no intervals
    df_period = df[['asset', 'day']]
    for i in range(2, 15):
        df_period[f"period{i}"] = df["day"] // i
    return df_period
def add_remainder(df):
    #no intervals
    df_remainder = df[['asset', 'day']]
    for i in range(2, 15):
        df_remainder[f"remainder{i}"] = df["day"] % i
    return df_remainder

def ctc_returns(df_market_a):
    #no intervals
    df_ctc = df_market_a[['asset', 'day']].reset_index(drop=True)
    for days in [1, 5, 10, 20]:
        df_ctc[f"ctc{days}"] = df_market_a['close'].pct_change(days).reset_index(drop=True)
    return df_ctc


def daily_volume_moving_ratio(df_market_a):
    #contains intervals
    #take df by asset type
    daily_volume_series = df_market_a.groupby(df_market_a['day'])['volume'].sum()
    df_movv = df_market_a[['asset', 'day']].drop_duplicates(subset=["asset", "day"], keep='last').reset_index(drop=True)
    df_movv["daily_volume_moving_ratio_5d"] = daily_volume_series / daily_volume_series.rolling(5).mean()
    df_movv["daily_volume_moving_ratio_10d"] = daily_volume_series / daily_volume_series.rolling(10).mean()
    df_movv["daily_volume_moving_ratio_20d"] = daily_volume_series / daily_volume_series.rolling(20).mean()
    return df_movv


#Talib Features **
def add_talib_features(df_market_a):
    #This function take reference to the "construct_talib_features" function and tailor to our dataframe settings
    #by creating asset, day, interval columns (prior), following code should be more understandable
    close_p = df_market_a[df_market_a['interval'] == 50]['close'].reset_index(drop=True)
    high_p = df_market_a.groupby(df_market_a['day'])['high'].max().reset_index(drop=True)
    low_p = df_market_a.groupby(df_market_a['day'])['low'].min().reset_index(drop=True)

    feature_df = df_market_a[['asset', 'day']].drop_duplicates(subset=["asset", "day"], keep='last').reset_index(drop=True)
    feature_df["SAR"] = ta.SAR(high_p, low_p, acceleration = 0, maximum = 0)
    feature_df["SAREXT"] = ta.SAREXT(high_p, low_p, startvalue = 0, offsetonreverse = 0, accelerationinitlong = 0, accelerationlong = 0, accelerationmaxlong = 0, accelerationinitshort = 0, accelerationshort = 0, accelerationmaxshort = 0)
    feature_df["RSI"] = ta.RSI(close_p, timeperiod = 14) - 50
    feature_df["HT_DCPERIOD"] = ta.HT_DCPERIOD(close_p)
    feature_df["HT_PHASOR_inphase"], feature_df["HT_PHASOR_quadrature"] = ta.HT_PHASOR(close_p)
    feature_df["HT_SINE_sine"], feature_df["HT_SINE_leadsine"] = ta.HT_SINE(close_p)
    return feature_df


In [ ]:
#Features in Consideration
def merge_columns(market, fundamental, returns=pd.DataFrame([0])):
    market = add_interval(market)
    fundamental = add_date(fundamental)
    df = pd.merge(market, fundamental, left_on=["asset", "day"], right_on=["asset", "day"])
    if returns.any()[0]:    #check if returns is a non-empty dataframe
        returns = add_date(returns)
        df = pd.merge(df, returns, left_on=["asset", "day"], right_on=["asset", "day"])
    # Note: This Step Will Mean that Metrics regarding
    return df
    # df = df.drop_duplicates(subset=["asset", "day"], keep='last').reset_index(drop=True)

def add_features(df):
    #THIS will return columns with only one set of ['asset', 'day'] (no intervals)
    df_close_only = df.drop_duplicates(subset=["asset", "day"], keep='last').reset_index(drop=True)
    df_features = df_close_only[['asset', 'day']]     #a df that contains the new feature to reduce loading time in each function call.

    #features not using intervals data
    df_features = pd.merge(df_features, add_period(df_close_only), left_on=["asset", "day"], right_on=["asset", "day"])
    df_features = pd.merge(df_features, add_remainder(df_close_only), left_on=["asset", "day"], right_on=["asset", "day"])
    df_features = pd.merge(df_close_only, df_features, left_on=["asset", "day"], right_on=["asset", "day"])

    #ctc returns
    ctc_features = pd.concat([ctc_returns(df_close_only[df_close_only['asset'] == i]) for i in range(54)]).reset_index(drop=True)
    df_features = pd.merge(df_features, ctc_features, left_on=["asset", "day"], right_on=["asset", "day"])

    #moving volume
    movv_features = pd.concat([daily_volume_moving_ratio(df[df['asset'] == i]) for i in range(54)]).reset_index(drop=True)
    df_features = pd.merge(df_features, movv_features, left_on=["asset", "day"], right_on=["asset", "day"])

    #talib features: iterate over each asset together, then merge by ["asset", "day"]
    talib_features = pd.concat([add_talib_features(df[df['asset'] == i]) for i in range(54)]).reset_index(drop=True)
    df_features = pd.merge(df_features, talib_features, left_on=["asset", "day"], right_on=["asset", "day"])
    return df_features

In [ ]:
df_temp = merge_columns(df_market, df_fund, df_return)
df = add_features(df_temp)
df

# Some Insights on Graphs

In [ ]:
asset_to_check = 43
df_temp = df[(df['asset'] == asset_to_check) & (df['day'] <= 1000)][['close', 'RSI']]
ax = df_temp[['close']].plot(figsize=(30, 15), color='red')
ax.set_xlabel("time")
ax.set_ylabel("close price")
#ax2 is to construct a second y axis on the graph (ax object)
ax2 = ax.twinx()
df_temp[['RSI']].plot(ax=ax2, style="--")
ax2.set_ylabel("RSI")

# Putting it to the Model (Reusing Part1's XGBoost (replace with lightgbm later?))

In [ ]:
def add_lag(df, target_map):
    for i in [700, 750, 800, 850, 900, 950]:   #needs to be larger than the 700 days in the future.
        df[f'lag{i}'] = (df.index - i*54).map(target_map)       #since each asset row is distanted at 54 rows (by 53)
    return df

In [ ]:
global returns_avg
features2 = ['turnoverRatio',
       'transactionAmount', 'pe_ttm', 'pe', 'pb', 'ps', 'pcf',
       'remainder7', 'remainder14',
       'ctc1', 'ctc5', 'ctc10',
       'ctc20', 'daily_volume_moving_ratio_5d',
       'daily_volume_moving_ratio_10d', 'daily_volume_moving_ratio_20d', 'SAR',
       'SAREXT', 'RSI', 'HT_DCPERIOD', 'HT_PHASOR_inphase',
       'HT_PHASOR_quadrature', 'HT_SINE_sine', 'HT_SINE_leadsine']
TARGET = 'return'
features2 = features2 + [f"lag{i}" for i in [700, 750, 800, 850, 900, 950]]
FEATURES4 = ['pcf', 'pe', 'pb', 'pe_ttm', 'ps', 'transactionAmount', 'turnoverRatio', 'money', 'open', 'close', 'high', 'low'] + [f"period{i}" for i in range(2, 15)] + [f"remainder{i}" for i in range(2, 15)]
FEATURES5 = FEATURES4 + [f"lag{i}" for i in [700, 750, 800, 850, 900, 950]]

In [ ]:
def train_predict(df_train, df_test, asset_type, features):
    # major changes to previous one: previous we have set arbitary where to split train and test set (e.g. 7:3)
    # this version make uses tss to have multiple folds of train and test sets (*without information leakage)
    lag_adjuster = df_train.shape[0]

    df_train = df_train[df_train['asset'] == asset_type]
    df_test = df_test[df_test['asset'] == asset_type]

    train_map = df_train['return'].to_dict()
    df_train = add_lag(df_train, train_map)
    df_train['isFuture'] = False
    df_test['isFuture'] = True
    df_test.index += lag_adjuster
    train_and_test = pd.concat([df_train, df_test])
    tat_map = train_and_test['return'].to_dict()
    train_and_test = add_lag(train_and_test, tat_map)
    df_test = train_and_test[train_and_test['isFuture'] == True].copy()

    x_all = df_train[features]
    y_all = df_train[TARGET]

    reg = xgb.XGBRegressor(n_estimators=2000,
                    booster="gbtree",
                    objective="reg:linear",
                    max_depth=2,            #high value leads to overfitting
                    learning_rate=0.4,
                    min_child_weight=6,             #higher value prevent overfitting (1000:700 ratio makes it easy to overfit)
                    subsample=1,
                    )
    
    reg.fit(x_all, y_all,
            eval_set=[(x_all, y_all)],
            verbose=20)
    
    df_test['prediction'] = reg.predict(df_test[features])
    df_progress = df_test[['prediction']]
    #return df_progress
    avg_adjusted_prediction = df_progress['prediction'] - (df_progress['prediction'].mean() - returns_avg[asset_type])
    return avg_adjusted_prediction  

In [ ]:
def train_predict_all():
    df_train = add_features(merge_columns(df_train_market, df_train_fund, df_train_returns))
    df_test = add_features(merge_columns(df_market, df_fund))
    global returns_avg
    returns_avg = [df_train[df_train['asset'] == i]['return'].mean() for i in range(54)]

    df_res = train_predict(df_train, df_test, 0, features_build)
    for i in range(1, 54):
        df_res = pd.concat((df_res, train_predict(df_train, df_test, i, features_build)))

    df_copy = df_res.sort_index()
    df_copy = df_copy.reset_index()
    dt_col = df_fund['date_time']
    df_copy['date_time'] = dt_col
    df_copy = df_copy[['date_time', 'prediction']]
    df_copy.columns = ['date_time', 'return']
    return df_copy

df_res = train_predict_all()
df_res
        

# Future Hacking and Hyper Parameter Tuning 👎

In [ ]:
df_2day_return = df_market[df_market['interval'] == 50]
df_2day_return = df_2day_return.reset_index()
df_2day_return = df_2day_return[['date_time', 'close']]
close_map = df_2day_return['close'].to_dict()

df_2day_return['close_2day_later'] = (df_2day_return.index + 2*54).map(close_map)
df_2day_return['2day_return'] = (df_2day_return['close_2day_later'] - df_2day_return['close']) / df_2day_return['close']
df_2day_return = df_2day_return.fillna(0)
real_2day_returns = df_2day_return['2day_return']

df_2day_return2 = df_2day_return.copy()
df_2day_return2['close_2day_later'] = (df_2day_return2.index - 2*54).map(close_map)
df_2day_return2['2day_return'] = (df_2day_return2['close_2day_later'] - df_2day_return2['close']) / df_2day_return2['close']
df_2day_return2 = df_2day_return2.fillna(0)
real_2day_returns2 = df_2day_return2['2day_return']

real_2day_returns

In [ ]:
print(df_res['return'].corr(real_2day_returns)) #should be the correct one.
print(df_res['return'].corr(real_2day_returns2))

# Makeshift Analysis on the return of the investment

A very basic computation of the average return among the asset chosen on each day, without catering transaction fees or closing fees.

In [ ]:
df_test = add_date(df_res)
df_test

In [ ]:
for d in range(1001, 1002):
    return_list = df_test[df_test['day'] == d]['return']
    returns_based_decision = return_list.apply(lambda x: 1 if x > 0.02 else 0)
    # returns_based_decision = list(returns_based_decision)
    print(f"This is Day: {d}, Returns are:")
    returns_based_weighting = returns_based_decision / returns_based_decision.sum()
    print(returns_based_weighting)